In [79]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Step 1:** Fetch data from kaggle using Big-query:

In [80]:
from google.cloud import bigquery
import bq_helper

# create a helper object for our bigquery dataset
historical_air_quality = bq_helper.BigQueryHelper(active_project="bigquery-public-data", \
                                                  dataset_name='epa_historical_air_quality')
historical_air_quality.head('air_quality_annual_summary')

Using Kaggle's public dataset BigQuery integration.


,state_code,county_code,site_num,parameter_code,poc,latitude,longitude,datum,parameter_name,sample_duration,...,seventy_five_percentile,fifty_percentile,ten_percentile,local_site_name,address,state_name,county_name,city_name,cbsa_name,date_of_last_change
0,23,003,1100,44201,1,46.696431,-68.033006,WGS84,Ozone,1 HOUR,...,0.043,0.040,0.035,MICMAC HEALTH DEPARTMENT,"8 NORTHERN ROAD, PRESQUE ISLE, ME 04769",Maine,Aroostook,Presque Isle,None,2022-04-05
1,23,003,1100,44201,1,46.696431,-68.033006,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.041,0.037,0.033,MICMAC HEALTH DEPARTMENT,"8 NORTHERN ROAD, PRESQUE ISLE, ME 04769",Maine,Aroostook,Presque Isle,None,2022-04-05
2,23,003,1100,44201,1,46.696431,-68.033006,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.041,0.037,0.033,MICMAC HEALTH DEPARTMENT,"8 NORTHERN ROAD, PRESQUE ISLE, ME 04769",Maine,Aroostook,Presque Isle,None,2022-04-05
3,23,003,1100,44201,1,46.696431,-68.033006,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.041,0.037,0.032,MICMAC HEALTH DEPARTMENT,"8 NORTHERN ROAD, PRESQUE ISLE, ME 04769",Maine,Aroostook,Presque Isle,None,2022-04-05
4,72,053,0003,81102,5,18.383983,-65.618888,WGS84,PM10 Total 0-10um STP,24 HOUR,...,29.000,19.000,9.000,Fajardo,"FAJARDO LIGHTHOUSE,FAJARDO",Puerto Rico,Fajardo,None,"San Juan-Carolina-Caguas, PR",2022-04-06


In [81]:
#historical_air_quality.table_schema('air_quality_annual_summary')

Add SQL select to filter out any unneeded attributes. Estimate size of resulting query to not exceed limits imposed by Kaggle on jupiter notebooks.

In [82]:
QUERY = """SELECT state_code, county_code, year, site_num, parameter_code, parameter_name, metric_used, units_of_measure, exceptional_data_count, first_max_value, arithmetic_mean 
         FROM `bigquery-public-data.epa_historical_air_quality.air_quality_annual_summary`
         WHERE parameter_name = 'Ozone' OR parameter_name = 'Carbon monoxide' OR parameter_name = 'Nitrogen dioxide (NO2)'
         OR parameter_name = 'PM10 Total 0-10um STP' OR parameter_name = 'Nitric oxide (NO)'
         OR parameter_name = 'PM2.5 - Local Conditions' OR parameter_name = 'Sulfur dioxide'"""

historical_air_quality.estimate_query_size(QUERY)

0.4459184901788831

Load raw data into Python dataframe.

In [83]:
df = historical_air_quality.query_to_pandas_safe(QUERY)
df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,state_code,county_code,year,site_num,parameter_code,parameter_name,metric_used,units_of_measure,exceptional_data_count,first_max_value,arithmetic_mean
0,23,003,2021,1100,44201,Ozone,Daily maxima of observed hourly values (betwee...,Parts per million,0,0.065,0.034665
1,23,003,2021,1100,44201,Ozone,Daily maximum of 8 hour running average of obs...,Parts per million,0,0.062,0.031335
2,23,003,2021,1100,44201,Ozone,Daily maximum of 8 hour running average of obs...,Parts per million,0,0.062,0.031335
3,72,061,2021,0005,81102,PM10 Total 0-10um STP,Daily Mean,Micrograms/cubic meter (25 C),0,74.000,24.173913
4,72,061,2021,0005,88101,PM2.5 - Local Conditions,Daily Mean,Micrograms/cubic meter (LC),0,23.300,7.565455


# **Step 2:** Clean data

For this step, FIPS column created for future joining.
Also, since some measurments were taken multople times for same parameter, using same metric, at the same site, but with different equipment, rows for that data were aggregated.

In [84]:
df["FIPS"] = df["state_code"].astype(str) + df["county_code"].astype(str)
df= df.sort_values(['state_code','county_code', 'year', 'parameter_code'])
df = df.groupby(['FIPS','state_code', 'county_code', 'year', 'parameter_code','parameter_name', 'metric_used', 'units_of_measure'], as_index=False).agg({'arithmetic_mean': 'mean', 'first_max_value':'max', 'exceptional_data_count':'sum'})
df.head()


,FIPS,state_code,county_code,year,parameter_code,parameter_name,metric_used,units_of_measure,arithmetic_mean,first_max_value,exceptional_data_count
0,01001,01,001,1980,42401,Sulfur dioxide,3-Hour block average of observed hourly values,Parts per billion,2.936202,33.300,0
1,01001,01,001,1980,42401,Sulfur dioxide,Daily Average of observed values,Parts per billion,2.903810,15.400,0
2,01001,01,001,1980,42401,Sulfur dioxide,Daily maximum 1-hour average,Parts per billion,8.876190,40.000,0
3,01001,01,001,1980,42401,Sulfur dioxide,Observed Values,Parts per billion,2.905290,40.000,0
4,01001,01,001,1980,44201,Ozone,Daily maxima of observed hourly values (betwee...,Parts per million,0.061478,0.125,0


Tidying data by setting attributes for same year into columns instead of having separate row for each parameter.

In [85]:
pivoted = df.pivot_table(index=['FIPS','state_code','county_code', 'year'],columns=['parameter_name', 'metric_used', 'units_of_measure'], values=['arithmetic_mean', 'first_max_value', 'exceptional_data_count'])#.squeeze()
pivoted.columns = pivoted.columns.map(' '.join)
pivoted.reset_index(inplace=True)
pivoted.reset_index(inplace=True)
pivoted

,index,FIPS,state_code,county_code,year,arithmetic_mean Carbon monoxide 8-Hour running average (end hour) of observed hourly values Parts per million,arithmetic_mean Carbon monoxide Observed Values Parts per million,arithmetic_mean Carbon monoxide Obseved hourly values Parts per million,arithmetic_mean Nitric oxide (NO) Observed Values Parts per billion,arithmetic_mean Nitrogen dioxide (NO2) Daily Maximum 1-hour average Parts per billion,...,first_max_value Ozone Daily maximum of 8-hour running average Parts per million,first_max_value PM10 Total 0-10um STP Daily Mean Micrograms/cubic meter (25 C),first_max_value PM10 Total 0-10um STP Observed Values Micrograms/cubic meter (25 C),first_max_value PM2.5 - Local Conditions Daily Mean Micrograms/cubic meter (LC),first_max_value PM2.5 - Local Conditions Observed Values Micrograms/cubic meter (LC),first_max_value PM2.5 - Local Conditions Quarterly Means of Daily Means Micrograms/cubic meter (LC),first_max_value Sulfur dioxide 3-Hour block average of observed hourly values Parts per billion,first_max_value Sulfur dioxide Daily Average of observed values Parts per billion,first_max_value Sulfur dioxide Daily maximum 1-hour average Parts per billion,first_max_value Sulfur dioxide Observed Values Parts per billion
0,0,01001,01,001,1980,NaN,NaN,NaN,NaN,NaN,...,0.096,NaN,NaN,NaN,NaN,NaN,33.3,15.4,40.0,40.0
1,1,01001,01,001,1981,NaN,NaN,NaN,NaN,NaN,...,0.103,NaN,NaN,NaN,NaN,NaN,45.0,17.2,100.0,100.0
2,2,01001,01,001,1982,NaN,NaN,NaN,NaN,NaN,...,0.111,NaN,NaN,NaN,NaN,NaN,23.3,14.5,30.0,30.0
3,3,01001,01,001,1989,NaN,NaN,NaN,NaN,NaN,...,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,01001,01,001,1990,NaN,NaN,NaN,NaN,NaN,...,0.086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40168,40168,CC011,CC,011,2002,NaN,NaN,NaN,NaN,16.149573,...,NaN,NaN,NaN,18.6,NaN,18.6,81.6,24.8,229.0,229.0
40169,40169,CC011,CC,011,2003,NaN,NaN,NaN,NaN,16.795252,...,NaN,NaN,NaN,34.5,NaN,34.5,87.0,19.8,168.0,168.0
40170,40170,CC011,CC,011,2004,NaN,NaN,NaN,NaN,12.731928,...,NaN,NaN,NaN,21.0,NaN,21.0,120.0,18.4,360.0,360.0
40171,40171,CC011,CC,011,2005,NaN,NaN,NaN,NaN,10.580858,...,NaN,NaN,NaN,18.0,NaN,18.0,72.3,12.8,149.0,149.0


List of columns:

In [86]:
pivoted.columns.values.tolist()

['index',
 'FIPS',
 'state_code',
 'county_code',
 'year',
 'arithmetic_mean Carbon monoxide 8-Hour running average (end hour) of observed hourly values Parts per million',
 'arithmetic_mean Carbon monoxide Observed Values Parts per million',
 'arithmetic_mean Carbon monoxide Obseved hourly values Parts per million',
 'arithmetic_mean Nitric oxide (NO) Observed Values Parts per billion',
 'arithmetic_mean Nitrogen dioxide (NO2) Daily Maximum 1-hour average Parts per billion',
 'arithmetic_mean Nitrogen dioxide (NO2) Observed Values Parts per billion',
 'arithmetic_mean Nitrogen dioxide (NO2) Observed values Parts per billion',
 'arithmetic_mean Ozone Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM) Parts per million',
 'arithmetic_mean Ozone Daily maximum of 8 hour running average of observed hourly values Parts per million',
 'arithmetic_mean Ozone Daily maximum of 8-hour running average Parts per million',
 'arithmetic_mean PM10 Total 0-10um STP Daily Mean Microgr

# **Sep 3:** Export data as CSV

In [87]:
pivoted.to_csv('air_quality.csv',index=False)